In [1]:
#Preliminaries
import os
from os import path
from os.path import isfile, join
from collections import defaultdict
import sys
import re
import json
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zjshe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zjshe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#File path used to read Corpus
dir_path = '/Users/zjshe/Documents/01. ECU MS CS/02. Spring 2022/02. CSCI 6030/03. Assignments/02. Programming Assignments/02-programming-assignment/All Files'
path = "C:\\Users\\zjshe\\Documents\\01. ECU MS CS\\02. Spring 2022\\02. CSCI 6030\\03. Assignments\\02. Programming Assignments\\02-programming-assignment\\corpus\\Good"
#Defining PorterStemmer
ps = PorterStemmer()

#List to store words
mylist = []

#Method for opening, reading, and encoding files from Corpus
def readFile(filename):
    with open(filename, "r", encoding='mac_roman') as f:
        dirtyTxt = f.read()
        
    f.close()

    return dirtyTxt




# Method for normalization and stemming of Corpus
def normalizeTxt(dirtyTxt):
    #text_lower = text.lower()
    
    #Tokenization
        tokenList = word_tokenize(dirtyTxt)
    
        tokenList = [word for word in tokenList if word.isalpha()]
    
    
        #Stemming
        stemedList = [ps.stem(word) for word in tokenList]
        return stemedList
        

#Method for building Positional Index
def posInd(path):

    posIndex = {}
    docDict = {}   
    docID = 0
    for root, subdirectories, files in os.walk(path):
        for file in files:

            filename = os.fsdecode(file)
            if(filename.endswith(".txt")):
                file = root + '\\' + filename
                tempTxt = readFile(file)
                cleanText = normalizeTxt(tempTxt)
                docDict[docID] = filename
                
                for position, word in enumerate(cleanText):
                    if word in posIndex:

                        posIndex[word][0] = posIndex[word][0] + 1

                        if docID in posIndex[word][1]:
                            posIndex[word][1][docID].append(position)
                             
                        else:
                            posIndex[word][1][docID] = [position]
                    else:
                         
                      
                        posIndex[word] = []
                        posIndex[word].append(1)
                        posIndex[word].append({})     
                        posIndex[word][1][docID] = [position]
            docID += 1

    return posIndex, docDict
 
#Compares two lists and takes the insterection of them
def intersect(list1, list2):
 
    
    temp = set(list2)
    lst = [value for value in list1 if value in temp]
    return lst

#Takes inputs and then performs a query
def PhraseQuery(queryWords, posIndex, i = 0):

    mainIndex = []

    for word in queryWords:
        try:
            mainIndex.append(posIndex[word])
        except KeyError:
            print("Word not found")
            return


    intersectResults = []

    count = 0

    for words in mainIndex:
        if count == 0:
            intersectResults = mainIndex[count][1].keys()
        else:
            intersectResults = intersect(intersectResults, mainIndex[count][1].keys())

        count += 1

    

    if i == 0:
        queryResult = []
        for docIDs in intersectResults:
            positionCheck = []
            for index, words in enumerate(mainIndex):
                tempInd = []
                tempInd = mainIndex[index][1].get(docIDs)
                    
                adjustedIndex = [x - index for x in tempInd]

                if index == 0:
                    positionCheck = adjustedIndex
                else: 
                    positionCheck = intersect(positionCheck, adjustedIndex)

            if positionCheck:
                queryResult.append(docIDs)
    else:
        queryResult = []
        for docIDs in intersectResults:
            positions1 = []
            positions2 = []
            for index, words in enumerate(mainIndex):
                tempIndex = []
                tempIndex = mainIndex[index][1].get(docIDs)
                kAdjustInd = [x - i for x in tempIndex]

                if index == 0:
                    positions1 = tempIndex
                else:
                    positions2 = kAdjustInd

                    for x in positions1:
                        for y in positions2:
                            if y <= x:
                                tempVal = x - y
                                if abs(tempVal) <= i:
                                    if docIDs not in queryResult:
                                        queryResult.append(docIDs)
                            else:
                                break

    return queryResult
        
#User interface that allows the user to select query type, enter query and assocaited parameters and provides results.     
def main():
    
    posIndex = {}
    docDict = {}
    posIndex, docDict = posInd(path)
    
    while(True):
        userInput = input("For proximity query enter 6. \nFor phrase query enter 9 \nTo Quit enter 0 \n")
        if int(userInput) == 6:
            inpt = input("Enter phrase to search: ")
            results = PhraseQuery(normalizeTxt(inpt), posIndex)
            if results:
                for x in results:
                    print(docDict.get(x))
                    
        elif int(userInput) == 9:
            inpt1 = input("Enter tokens to search: ")
            inpt2 = int(input("Enter distance between values as a whole numbers: "))
            results = PhraseQuery(normalizeTxt(inpt1), posIndex, int(inpt2))
            if results:
                for x in results:
                    print(docDict.get(x))
            
        elif int(userInput) == 0:
            exit()
                          
        else:
            exit()

                    
        
if __name__ == "__main__":
    main()

For proximity query enter 6. 
For phrase query enter 9 
To Quit enter 0 
0


SystemExit: 

C:\Users\zjshe\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
